In [3]:
//@file:Repository("*mavenLocal")
// @file:DependsOn("org.jetbrains.kotlin.tensorflow:Kotof:0.0.4")
@file:Repository("https://kotlin.bintray.com/kotlin-datascience")
@file:DependsOn("org.jetbrains.kotlin.kotlin-dl:api:0.0.1")

In [14]:
import api.keras.Sequential
import api.keras.activations.Activations
import api.keras.dataset.ImageDataset
import api.keras.initializers.Constant
import api.keras.initializers.HeNormal
import api.keras.initializers.Zeros
import api.keras.layers.Dense
import api.keras.layers.Flatten
import api.keras.layers.Input
import api.keras.layers.twodim.Conv2D
import api.keras.layers.twodim.ConvPadding
import api.keras.layers.twodim.MaxPool2D
import api.keras.loss.LossFunctions
import api.keras.metric.Metrics
import api.keras.optimizers.Adam
import datasets.*

val EPOCHS = 2
val TRAINING_BATCH_SIZE = 500
val TEST_BATCH_SIZE = 1000
val NUM_CHANNELS = 1L
val IMAGE_SIZE = 28L
val SEED = 12L

In [15]:
val model = Sequential.of(
    Input(
        IMAGE_SIZE,
        IMAGE_SIZE,
        NUM_CHANNELS
    ),
    Conv2D(
        filters = 32,
        kernelSize = longArrayOf(5, 5),
        strides = longArrayOf(1, 1, 1, 1),
        activation = Activations.Relu,
        kernelInitializer = GlorotNormal(SEED),
        biasInitializer = Zeros(),
        padding = ConvPadding.SAME
    ),
    MaxPool2D(
        poolSize = intArrayOf(1, 2, 2, 1),
        strides = intArrayOf(1, 2, 2, 1)
    ),
    Conv2D(
        filters = 64,
        kernelSize = longArrayOf(5, 5),
        strides = longArrayOf(1, 1, 1, 1),
        activation = Activations.Relu,
        kernelInitializer = GlorotNormal(SEED),
        biasInitializer = Zeros(),
        padding = ConvPadding.SAME
    ),
    MaxPool2D(
        poolSize = intArrayOf(1, 2, 2, 1),
        strides = intArrayOf(1, 2, 2, 1)
    ),
    Flatten(),
    Dense(
        outputSize = 512,
        activation = Activations.Relu,
        kernelInitializer = GlorotNormal(SEED),
        biasInitializer = Constant(0.1f)
    ),
    Dense(
        outputSize = 10,
        activation = Activations.Linear,
        kernelInitializer = GlorotNormal(SEED),
        biasInitializer = Constant(0.1f)
    )
)

In [16]:
    val (train, test) = ImageDataset.createTrainAndTestDatasets(
        TRAIN_IMAGES_ARCHIVE,
        TRAIN_LABELS_ARCHIVE,
        TEST_IMAGES_ARCHIVE,
        TEST_LABELS_ARCHIVE,
        10,
        ::extractImages,
        ::extractLabels
    )

Extracting 60000 images of 28x28 from train-images-idx3-ubyte.gz
Extracting 60000 labels from train-labels-idx1-ubyte.gz
Extracting 10000 images of 28x28 from t10k-images-idx3-ubyte.gz
Extracting 10000 labels from t10k-labels-idx1-ubyte.gz


In [17]:
model.compile(optimizer = Adam(), loss = LossFunctions.SOFT_MAX_CROSS_ENTROPY_WITH_LOGITS)

model.fit(dataset = train, epochs = EPOCHS, batchSize = TRAINING_BATCH_SIZE, verbose = true)

val accuracy = model.evaluate(dataset = test, batchSize = TEST_BATCH_SIZE)

println("Accuracy: $accuracy")

model.close()

16:07:09.723 [main] DEBUG api.keras.Sequential - api.keras.layers.twodim.Conv2D@3b32f0b4 [-1, 28, 28, 32]
16:07:09.728 [main] DEBUG api.keras.Sequential - api.keras.layers.twodim.MaxPool2D@6c54d0dc [-1, 14, 14, 32]
16:07:09.731 [main] DEBUG api.keras.Sequential - api.keras.layers.twodim.Conv2D@5b5f06bb [-1, 14, 14, 64]
16:07:09.733 [main] DEBUG api.keras.Sequential - api.keras.layers.twodim.MaxPool2D@7c74ef4a [-1, 7, 7, 64]
16:07:09.734 [main] DEBUG api.keras.Sequential - api.keras.layers.Flatten@54519d74 [3136]
16:07:09.738 [main] DEBUG api.keras.Sequential - api.keras.layers.Dense@1ec96512 [512]
16:07:09.740 [main] DEBUG api.keras.Sequential - api.keras.layers.Dense@3d00c2ae [10]
16:07:09.751 [main] DEBUG api.keras.Sequential - Initialization of TensorFlow Graph variables
16:07:10.439 [main] DEBUG api.keras.Sequential - Batch stat: { lossValue: 2.3205056 metricValue: 0.102 }
16:07:10.719 [main] DEBUG api.keras.Sequential - Batch stat: { lossValue: 2.1706195 metricValue: 0.266 }
16:07

In [18]:
model.summary()

16:09:05.196 [main] INFO  api.keras.Sequential - =================================================================
16:09:05.198 [main] INFO  api.keras.Sequential - Model: Sequential
16:09:05.200 [main] INFO  api.keras.Sequential - _________________________________________________________________
16:09:05.201 [main] INFO  api.keras.Sequential - Layer (type)                 Output Shape              Param #   
16:09:05.202 [main] INFO  api.keras.Sequential - =================================================================
16:09:05.205 [main] INFO  api.keras.Sequential - conv2d_1(Conv2D)             [-1, 28, 28, 32]          832
16:09:05.207 [main] INFO  api.keras.Sequential - _________________________________________________________________
16:09:05.208 [main] INFO  api.keras.Sequential - maxpool2d_2(MaxPool2D)       [-1, 14, 14, 32]          0
16:09:05.211 [main] INFO  api.keras.Sequential - _________________________________________________________________
16:09:05.213 [main] INFO  api

[conv2d_1(Conv2D)             [-1, 28, 28, 32]          832, maxpool2d_2(MaxPool2D)       [-1, 14, 14, 32]          0, conv2d_3(Conv2D)             [-1, 14, 14, 64]          51264, maxpool2d_4(MaxPool2D)       [-1, 7, 7, 64]            0, flatten_5(Flatten)           [3136]                    0, dense_6(Dense)               [512]                     1606144, dense_7(Dense)               [10]                      5130]